In [1]:
import numpy as np
import pandas as pd

In [27]:
#행정동 = '장위2동' # 3개 -> 1
#행정동 = '돈암1동' # 5개 -> 2
행정동 = '망우3동' # 4개 -> 1
#행정동 = '중곡3동' # 2개 -> 1

In [28]:
입지후보지 = pd.read_csv('수정_최종후보입지.csv', encoding='EUC-KR')
입지후보지 = 입지후보지[입지후보지['행정동'] == 행정동]
후보지_points = np.array([list(i) for i in zip(입지후보지['x좌표'], 입지후보지['y좌표'])])

버스 = pd.read_csv('입지3_버스.csv', encoding='utf-8')
#버스 = 버스[버스['행정동'].isin(행정동리스트)]
버스_points = np.array([list(i) for i in zip(버스['X좌표'], 버스['Y좌표'])])

지하철 = pd.read_csv('입지3_지하철.csv', encoding='utf-8')
#지하철 = 지하철[지하철['행정동'].isin(행정동리스트)]
지하철_points = np.array([list(i) for i in zip(지하철['경도'], 지하철['위도'])])

주차장 = pd.read_csv('입지3_주차장.csv', encoding='utf-8')
#주차장 = 주차장[주차장['행정동'].isin(행정동리스트)]
주차장_points = np.array([list(i) for i in zip(주차장['경도'], 주차장['위도'])])

주택 = pd.read_csv('입지3_주택.csv', encoding='utf-8')
#주택 = 주택[주택['행정동'].isin(행정동리스트)]
주택_points = np.array([list(i) for i in zip(주택['위도'], 주택['경도'])])   # 위도 경도 바뀜

In [29]:
X = list(버스['X좌표']) + list(지하철['경도']) + list(주차장['경도']) + list(주택['위도'])     # 주택 위도 경도 이름 바뀜
Y = list(버스['Y좌표']) + list(지하철['위도']) + list(주차장['위도']) + list(주택['경도'])

In [30]:
points = np.array([list(i) for i in zip(X, Y)])

In [31]:
전체w = points.shape[0]
버스w = 버스.shape[0]
지하철w = 지하철.shape[0]
주차장w = 주차장.shape[0]
주택w = 주택.shape[0]

In [32]:
# 가중치 by AHP 분석 (데이터 불균형 * 통행수단 선호도) 
m1 = (전체w-버스w)/전체w * 0.278 
m2 = (전체w-지하철w)/전체w * 0.193 
m3 = (전체w-주차장w)/전체w * 0.136 
m4 = (전체w-주택w)/전체w * 0.392 

In [33]:
from haversine import haversine

havers = []
for i in 후보지_points:
    site = []
    for j in 버스_points:
        site.append(haversine(i,j))
    havers.append(site)

# 버스

In [34]:
from pulp import *

# 후보지와 버스정류장 간 거리행렬 생성
location = list(입지후보지['시설명'])
location2 = list(버스['정류소명'])

havers_D = dict(zip(location, [dict(zip(location2, i)) for i in havers]))
D = havers_D

In [35]:
#col = list(D.keys())
D = pd.DataFrame(D)
D.head()

,꽃동산,달동산,샛 별
IBK기업은행면목동지점,1.585768,1.173554,1.797810
KT중랑지점,2.315059,1.883658,2.510081
KT중랑지점사거리,2.027686,1.594797,2.220530
건영아파트,0.811775,0.681454,0.793437
경남아파트,1.227840,1.537068,0.991815


In [36]:
# #col = D.columns
최소값 = list(D.min(axis=1))

In [37]:
# 버스
idx = D.index
col = D.columns
lidx = len(idx)
lcol = len(col)

for i in range(lidx):
    for j in range(lcol):
        if D.loc[idx[i], col[j]] == 최소값[i]:
            D.loc[idx[i], col[j]] = m1
        else:
            D.loc[idx[i], col[j]] = 0

# 지하철

In [38]:
from pulp import *

# 후보지와 버스정류장 간 거리행렬 생성
location = list(입지후보지['시설명'])
location2 = list(지하철['전철역명'])

havers_D2 = dict(zip(location, [dict(zip(location2, i)) for i in havers]))
D2 = havers_D2

In [39]:
#col = list(D.keys())
D2 = pd.DataFrame(D2)
D2.head()

,꽃동산,달동산,샛 별
망우역,1.209662,0.801779,1.423176
면목역,1.227840,1.537068,0.991815
사가정역,0.648044,0.450959,0.691154
상봉역,0.795960,0.399858,1.011516
양원역,1.725624,1.372052,1.957082


In [40]:
# #col = D.columns
최소값 = list(D2.min(axis=1))

In [41]:
# 지하철
idx = D2.index
col = D2.columns
lidx = len(idx)
lcol = len(col)

for i in range(lidx):
    for j in range(lcol):
        if D2.loc[idx[i], col[j]] == 최소값[i]:
            D2.loc[idx[i], col[j]] = m2
        else:
            D2.loc[idx[i], col[j]] = 0

# 주차장

In [42]:
from pulp import *

# 후보지와 버스정류장 간 거리행렬 생성
location = list(입지후보지['시설명'])
location2 = list(주차장['주차장이름'])

havers_D3 = dict(zip(location, [dict(zip(location2, i)) for i in havers]))
D3 = havers_D3

In [43]:
#col = list(D.keys())
D3 = pd.DataFrame(D3)
D3.head()

,꽃동산,달동산,샛 별
그린존 망우3동 공영주차장,1.471598,1.088770,1.567169
그린존 상봉2동 공영주차장,0.566058,0.659240,0.464692
금란주차장,1.353404,1.044237,1.589359
까치공원공영주차장,0.648044,0.450959,0.691154
노상공영주차장,1.316882,0.976005,1.384849


In [44]:
# #col = D.columns
최소값 = list(D3.min(axis=1))

In [45]:
# 주차장
idx = D3.index
col = D3.columns
lidx = len(idx)
lcol = len(col)

for i in range(lidx):
    for j in range(lcol):
        if D3.loc[idx[i], col[j]] == 최소값[i]:
            D3.loc[idx[i], col[j]] = m3
        else:
            D3.loc[idx[i], col[j]] = 0

# 주택

In [46]:
from pulp import *

# 후보지와 버스정류장 간 거리행렬 생성
location = list(입지후보지['시설명'])
location2 = list(주택['단지명'])

havers_D4 = dict(zip(location, [dict(zip(location2, i)) for i in havers]))
D4 = havers_D4

In [47]:
#col = list(D.keys())
D4 = pd.DataFrame(D4)
D4.head()

,꽃동산,달동산,샛 별
신내10,1.227840,1.537068,0.991815
신내12,0.992206,0.591990,1.116943
신내데시앙,1.737819,1.299816,1.905318
한양수자인 사가정파크(면목1),0.648044,0.450959,0.691154
현대프레미어스엠코(상봉8),1.442826,1.125971,1.678357


In [48]:
# #col = D.columns
최소값 = list(D4.min(axis=1))

In [49]:
# 주택
idx = D4.index
col = D4.columns
lidx = len(idx)
lcol = len(col)

for i in range(lidx):
    for j in range(lcol):
        if D4.loc[idx[i], col[j]] == 최소값[i]:
            D4.loc[idx[i], col[j]] = m4
        else:
            D4.loc[idx[i], col[j]] = 0

# Final

In [50]:
D_final = pd.concat([D, D2, D3, D4])

In [51]:
D_final.sum()

꽃동산      0.414201
달동산     10.491134
샛  별     3.088134
dtype: float64

* (3개 중 1위) 장위2동 - 장위2동 감나무쉼터 - 127.051625	37.614919
* (4개 중 1위) 돈암1동 - 돈암1동 마을마당 - 127.021175	37.597746
* (4개 중 2위) 돈암1동 - 새소리 - 127.025559	37.600827
* (3개 중 1위) 망우3동 - 달동산 - 127.0952	37.59217
* (2개 중 1위) 중곡3동 - 배나무터공원 - 127.0802718	37.56833955